In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import os
from tensorflow import convert_to_tensor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN


C:\Users\Plamen\AppData\Local\Temp\ipykernel_19072\1389482001.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def generate_time_series( n_steps):
    time = np.linspace(0,1,n_steps)
    series = 0.5 * np.sin(time)
    series += 0.1 * (np.random.rand(n_steps) - 0.5)
    return series[...,np.newaxis].astype(np.float32)

In [3]:
generate_time_series(20)

array([[-0.02789092],
       [ 0.06009863],
       [ 0.02517983],
       [ 0.10482239],
       [ 0.07959697],
       [ 0.11821268],
       [ 0.16242681],
       [ 0.21687399],
       [ 0.21058604],
       [ 0.23838317],
       [ 0.24191654],
       [ 0.29824594],
       [ 0.33103734],
       [ 0.2692502 ],
       [ 0.3387884 ],
       [ 0.3736774 ],
       [ 0.41589   ],
       [ 0.3509717 ],
       [ 0.45465156],
       [ 0.37436953]], dtype=float32)

In [4]:
def forecast(model, start, y_all, sequence_length):
    N = len(y_all)-start
    A = y_all[start-sequence_length:start].to_numpy().reshape(-1,1).T
    y_pred = list()
    for i in range(N):
        z = A
        z = convert_to_tensor(z)
        r = model(z)
        y_pred.append(r[0,0])
        A = np.append(A[0,1:], r[0,0]).reshape(-1,1).T
    y_pred = pd.Series(y_pred,index=y_all.index[start:])
    return y_pred

# Load the data

In [5]:
def MAE(x):
    return np.abs(x).mean() 

def RMSE(x):
    return np.sqrt( np.square(x).mean() )

def plot_decomp(y_hist,y_future,D):
    plt.figure(figsize=(14,4))
    plt.plot(D['residue'],'k')
    plt.plot(D['pred_error'],'m')

    plt.figure(figsize=(16,5))
    plt.plot(y_hist,'k',label='data',linewidth=1)
    plt.plot(D['season_and_trend'],label='model',color='royalblue',linewidth=3)
    plt.plot(D['yhat'],label='forecast',color='limegreen',linewidth=3)
    plt.plot(y_future,'k',alpha=0.5,linewidth=1)

In [6]:
with open('decomp.pkl', 'rb') as file:
    X = pickle.load(file)

history = X['history']
y_hist = history['y_hist']
future = X['future']
y_future = future['y_future']
y_all = y_hist.append(y_future)

Nhist = len(y_hist)

time_all = y_all.index
time_hist = y_hist.index
time_future = y_future.index


decomp = {}
decomp['season_and_trend'] = history['t_and_s']
decomp['residue'] = y_hist - history['t_and_s']
decomp['MAEm'] = MAE(decomp['residue'])
decomp['RMSEm'] = RMSE(decomp['residue'])
decomp['y_future'] = future['y_future']
decomp['yhat'] = future['y_hat']
decomp['pred_error'] = decomp['y_future'] - decomp['yhat']
decomp['MAEp'] = MAE(decomp['pred_error'])
decomp['RMSEp'] = RMSE(decomp['pred_error'])

FileNotFoundError: [Errno 2] No such file or directory: 'decomp.pkl'

In [ ]:
plot_decomp(y_hist,y_future,decomp)

# Train test data

In [ ]:
sequence_length = 50
lookahead = 1
delay = sequence_length + lookahead - 1

train_data = y_hist
train_target = np.zeros(y_hist.shape)
train_target[:-delay] = y_hist[delay:]
train_target = pd.Series(train_target,index = train_data.index)

train_dataset = timeseries_dataset_from_array(
    data = train_data,
    targets= train_target,
    sequence_length = sequence_length,
    batch_size = 1 )

test_data = y_all[Nhist-delay:]
test_target = np.zeros(test_data.shape)
test_target[:-delay] = y_all[Nhist:]
test_target = pd.Series(test_target,index = test_data.index)

test_dataset = timeseries_dataset_from_array(
    data = test_data,
    targets= test_target,
    sequence_length = sequence_length,
    batch_size = 1 )

# Dense neural network

In [ ]:
inputs = keras.Input(shape=(sequence_length, 1))
x = layers.Flatten()(inputs)
x = layers.Dense(5, activation="relu")(x)
outputs = layers.Dense(1)(x)
model_dense = keras.Model(inputs, outputs)
model_dense.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model_dense.fit(train_dataset,epochs=10)

In [ ]:
y_hist_hat = model_dense.predict(train_dataset)
y_future_hat = model_dense.predict(test_dataset)
t_hist_hat = time_hist[delay-1:(delay+len(y_hist_hat))]
t_future_hat = time_all[-len(y_future_hat):]

# training and testing error

In [ ]:
model_dense.evaluate(train_dataset)

In [ ]:
model_dense.evaluate(test_dataset)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(y_hist,marker='.',linewidth=2)
plt.plot(y_future,marker='.',linewidth=2)
plt.plot(t_hist_hat,y_hist_hat,linewidth=2,marker='.',color='r')
plt.plot(t_future_hat,y_future_hat,linewidth=2,marker='.',color='g')
plt.xlim(100,12000)

# Forecast

In [ ]:
start = 1020
y_pred = forecast(model_dense, start, y_all, sequence_length)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(y_hist,marker='.',linewidth=2)
plt.plot(y_future,marker='.',linewidth=2)
plt.plot(y_all[start-sequence_length:start],marker='.',linewidth=2)
plt.plot(y_pred,marker='.',linewidth=2)
plt.xlim(100,12000)

# RNN

In [ ]:
model_rnn = keras.models.Sequential([
    SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
    SimpleRNN(20, return_sequences=True),
    SimpleRNN(1)
])

model_rnn.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model_rnn.fit(train_dataset,epochs=10)

In [ ]:
inputs = keras.Input(shape=(sequence_length, 1))
x = layers.SimpleRNN(3, return_sequences=True)(inputs)
outputs = layers.Dense(1)(x)
model_rnn = keras.Model(inputs, outputs)
model_rnn.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model_rnn.fit(train_dataset,epochs=10)

In [ ]:
# y_future_hat = model_rnn.predict(test_dataset)


In [ ]:
for seq, trgt in test_dataset:
    print(seq)
    print(model_rnn.predict(seq))
    break

In [ ]:
y_pred = forecast(model_rnn, start, y_all, sequence_length)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(y_hist,marker='.',linewidth=2)
plt.plot(y_future,marker='.',linewidth=2)
plt.plot(y_all[start-sequence_length:start],marker='.',linewidth=2)
plt.plot(y_pred,marker='.',linewidth=2)
plt.plot(t_future_hat,y_future_hat,linewidth=2,marker='.',color='g')

plt.xlim(100,12000)

In [ ]:
model_rnn.layers


# LSTM

In [ ]:
inputs = keras.Input(shape=(sequence_length, 1))
x = layers.LSTM(20)(inputs)
outputs = layers.Dense(1)(x)
model_lstm = keras.Model(inputs, outputs)
model_lstm.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model_lstm.fit(train_dataset,epochs=10)

In [ ]:
y_pred = forecast(model_lstm, start, y_all, sequence_length)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(y_hist,marker='.',linewidth=2)
plt.plot(y_future,marker='.',linewidth=2)
plt.plot(y_all[start-sequence_length:start],marker='.',linewidth=2)
plt.plot(y_pred,marker='.',linewidth=2)
plt.xlim(100,12000)